In [1]:
import pandas as pd
import numpy as np
import tqdm
import os

import logging
logging.basicConfig(format='%(asctime)s [%(levelname)s] %(message)s',
                    level=logging.INFO)

import nltk
from nltk import FreqDist

from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup
import lxml
import re

In [2]:
df_scores = pd.read_csv("/Users/shashanksingh/Desktop/air_pollution/results/tfidf_scores.csv")

In [3]:
df_scores.head()

,Unnamed: 0,phrase,mean_score
0,1664359,pollution control board,0.019531
1,2122725,state pollution control,0.011684
2,1705359,prevention control pollution,0.010385
3,529159,control pollution act,0.009576
4,1213057,learned counsel appearing,0.008518


In [4]:
df_scores["global_warming"] = df_scores["phrase"].apply(lambda x:1 if "global warming" in x else 0)

In [5]:
df_scores["global_warming"].describe()

count    2.446760e+06
mean     1.798297e-05
std      4.240595e-03
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: global_warming, dtype: float64

In [6]:
df_gw = df_scores[df_scores["global_warming"]==1]

In [7]:
len(df_scores)

2446760

In [8]:
len(df_gw)

44

In [9]:
df_gw.head()

,Unnamed: 0,phrase,mean_score,global_warming
24885,2316331,upon global warming,0.000103,1
35761,946806,global warming areas,0.000077,1
309582,946811,global warming environment,0.000020,1
310070,1241875,like global warming,0.000020,1
310123,946824,global warming recurring,0.000020,1


In [10]:
df_scores["carbon"] = df_scores["phrase"].apply(lambda x:1 if "carbon" in x else 0)
df_carbon = df_scores[df_scores["carbon"]==1]
df_carbon.describe()

,Unnamed: 0,mean_score,global_warming,carbon
count,5.040000e+02,5.040000e+02,504.000000,504.0
mean,9.206362e+05,1.236211e-05,0.001984,1.0
std,6.565283e+05,1.435810e-05,0.044544,0.0
min,1.708800e+04,4.111236e-07,0.000000,1.0
25%,3.281488e+05,5.505241e-06,0.000000,1.0
50%,7.755955e+05,9.147684e-06,0.000000,1.0
75%,1.374595e+06,1.633058e-05,0.000000,1.0
max,2.435204e+06,1.655069e-04,1.000000,1.0


In [11]:
df_carbon.head()

,Unnamed: 0,phrase,mean_score,global_warming,carbon
12568,1630116,philips carbon black,0.000166,0,1
15646,1389170,ms philips carbon,0.000137,0,1
18845,328077,carbon black ltd,0.000116,0,1
38091,1032228,hydro carbons energy,0.000074,0,1
38094,218498,atomic hydro carbons,0.000074,0,1


In [12]:
df_scores["sulphur"] = df_scores["phrase"].apply(lambda x:1 if "sulphur" in x else 0)
df_sulphur = df_scores[df_scores["sulphur"]==1]
df_sulphur.describe()

,Unnamed: 0,mean_score,global_warming,carbon,sulphur
count,6.280000e+02,6.280000e+02,628.0,628.000000,628.0
mean,1.479324e+06,7.028837e-06,0.0,0.009554,1.0
std,7.154888e+05,8.900200e-06,0.0,0.097355,0.0
min,5.739000e+03,4.111236e-07,0.0,0.000000,1.0
25%,8.003438e+05,3.779582e-06,0.0,0.000000,1.0
50%,1.644994e+06,6.007854e-06,0.0,0.000000,1.0
75%,2.179437e+06,7.801567e-06,0.0,0.000000,1.0
max,2.417561e+06,1.654797e-04,0.0,1.000000,1.0


In [13]:
df_sulphur.head()

,Unnamed: 0,phrase,mean_score,global_warming,carbon,sulphur
12572,2179517,sulphuric acid plant,0.000165,0,0,1
55232,463792,concentration sulphur dioxide,0.000056,0,0,1
69113,2179519,sulphuric acid plants,0.000048,0,0,1
79898,2179433,sulphur dioxide oxides,0.000045,0,0,1
86849,1171723,kerosene sulphuric acid,0.000042,0,0,1


In [14]:
df_scores["sulfur"] = df_scores["phrase"].apply(lambda x:1 if "sulfur" in x else 0)
df_sulfur = df_scores[df_scores["sulfur"]==1]
df_sulfur.describe()

,Unnamed: 0,mean_score,global_warming,carbon,sulphur,sulfur
count,2.400000e+01,24.000000,24.0,24.0,24.0,24.0
mean,1.264353e+06,0.000006,0.0,0.0,0.0,1.0
std,7.650285e+05,0.000004,0.0,0.0,0.0,0.0
min,2.025300e+04,0.000002,0.0,0.0,0.0,1.0
25%,6.590022e+05,0.000002,0.0,0.0,0.0,1.0
50%,1.119368e+06,0.000003,0.0,0.0,0.0,1.0
75%,2.179242e+06,0.000008,0.0,0.0,0.0,1.0
max,2.433926e+06,0.000013,0.0,0.0,0.0,1.0


In [15]:
df_sulfur.head()

,Unnamed: 0,phrase,mean_score,global_warming,carbon,sulphur,sulfur
585106,2179246,sulfuric acid name,0.000013,0,0,0,1
585202,20253,acetic detergent sulfuric,0.000013,0,0,0,1
585242,659526,detergent sulfuric acid,0.000013,0,0,0,1
753635,170466,application wz sulfur,0.000011,0,0,0,1
753710,2179244,sulfur oxides oxides,0.000011,0,0,0,1


In [16]:
df_scores["nitrogen"] = df_scores["phrase"].apply(lambda x:1 if "nitrogen" in x else 0)
df_nitrogen = df_scores[df_scores["nitrogen"]==1]
df_nitrogen.describe()

,Unnamed: 0,mean_score,global_warming,carbon,sulphur,sulfur,nitrogen
count,2.010000e+02,2.010000e+02,201.0,201.000000,201.000000,201.0,201.0
mean,1.327471e+06,7.806776e-06,0.0,0.049751,0.044776,0.0,1.0
std,5.099758e+05,6.086607e-06,0.0,0.217973,0.207328,0.0,0.0
min,1.113660e+05,4.111236e-07,0.0,0.000000,0.000000,0.0,1.0
25%,1.016701e+06,3.976185e-06,0.0,0.000000,0.000000,0.0,1.0
50%,1.431922e+06,4.788884e-06,0.0,0.000000,0.000000,0.0,1.0
75%,1.533708e+06,1.046489e-05,0.0,0.000000,0.000000,0.0,1.0
max,2.442725e+06,4.483516e-05,0.0,1.000000,1.000000,0.0,1.0


In [17]:
df_nitrogen.head()

,Unnamed: 0,phrase,mean_score,global_warming,carbon,sulphur,sulfur,nitrogen
79899,672896,dioxide oxides nitrogen,0.000045,0,0,0,0,1
144700,1533723,oxides nitrogen nox,0.000032,0,0,0,0,1
170207,1431945,nitrogen oxygen minor,0.000030,0,0,0,0,1
170209,301313,breathe mixture nitrogen,0.000030,0,0,0,0,1
170211,1366097,mixture nitrogen oxygen,0.000030,0,0,0,0,1


In [18]:
df_scores["ozone"] = df_scores["phrase"].apply(lambda x:1 if "ozone" in x else 0)
df_ozone = df_scores[df_scores["ozone"]==1]
df_ozone.describe()

,Unnamed: 0,mean_score,global_warming,carbon,sulphur,sulfur,nitrogen,ozone
count,3.600000e+01,36.000000,36.0,36.0,36.0,36.0,36.0,36.0
mean,1.308272e+06,0.000009,0.0,0.0,0.0,0.0,0.0,1.0
std,5.634488e+05,0.000005,0.0,0.0,0.0,0.0,0.0,0.0
min,2.182780e+05,0.000003,0.0,0.0,0.0,0.0,0.0,1.0
25%,7.200210e+05,0.000005,0.0,0.0,0.0,0.0,0.0,1.0
50%,1.533848e+06,0.000006,0.0,0.0,0.0,0.0,0.0,1.0
75%,1.563712e+06,0.000010,0.0,0.0,0.0,0.0,0.0,1.0
max,2.444827e+06,0.000020,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
df_ozone.head()

,Unnamed: 0,phrase,mean_score,global_warming,carbon,sulphur,sulfur,nitrogen,ozone
310045,218278,atmosphere effect ozone,0.000020,0,0,0,0,0,1
310112,1533854,ozone layer threatens,0.000020,0,0,0,0,0,1
310226,740160,effect ozone layer,0.000020,0,0,0,0,0,1
397202,298190,borne pollutants ozone,0.000017,0,0,0,0,0,1
397890,1533845,ozone co co,0.000017,0,0,0,0,0,1


In [ ]:
df_scores["pollution"] = df_scores["phrase"].apply(lambda x:1 if "pollution" in x else 0)
df_sulfur = df_scores[df_scores["sulfur"]==1]
df_sulfur.describe()

In [21]:
df_scores.head(50)

,Unnamed: 0,phrase,mean_score,global_warming,carbon,sulphur,sulfur,nitrogen,ozone
0,1664359,pollution control board,0.019531,0,0,0,0,0,0
1,2122725,state pollution control,0.011684,0,0,0,0,0,0
2,1705359,prevention control pollution,0.010385,0,0,0,0,0,0
3,529159,control pollution act,0.009576,0,0,0,0,0,0
4,1213057,learned counsel appearing,0.008518,0,0,0,0,0,0
5,90733,air prevention control,0.006804,0,0,0,0,0,0
6,2377093,water prevention control,0.006534,0,0,0,0,0,0
7,1408082,national green tribunal,0.006312,0,0,0,0,0,0
8,1213342,learned counsel petitioner,0.006267,0,0,0,0,0,0
9,392127,civil writ petition,0.006111,0,0,0,0,0,0
